In [3]:
import torch
import torch.nn
import torch.optim

import GraphX as gx
import ConnectionGraphX as cgx
import numpy as np
import networkx as nx
import scipy as sp

import ConnectionNetworkX as cnx

import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm


In [4]:
'''
phi,c: |V|d x 1
B: |E|d x |V|d      Connection incidence matrix
w: |E| x 1          Edge weights
c: |V|d x 1         c = alpha-beta; i.e. difference of densities
'''
def loss_fn(phi, B, w, c, alpha):
    loss0 = -torch.sum(phi*c)
    
    loss1 = torch.matmul(B, phi).reshape((w.shape[0],-1))
    loss1 = torch.linalg.norm(loss1, dim=1)
    loss1 = loss1 - w
    loss1 = torch.nn.ReLU()(loss1)
    loss1 = torch.sum(loss1**2)
    
    loss = loss0 + (0.5/alpha)*loss1
    return loss

In [5]:
def optimize(B, w, c, alpha, learning_rate, n_epochs, phi0 = None, print_freq=10):
    if phi0 is None:
        phi = torch.randn(B.shape[1], 1, requires_grad=True)
    else:
        phi = torch.tensor(phi0, requires_grad=True)
    optimizer = torch.optim.Adam([phi], lr=learning_rate)
    for epoch in range(n_epochs):
        # Compute loss
        loss = loss_fn(phi, B, w, c, alpha)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % print_freq == 0:
            print(f"epoch: {epoch}, loss: {loss:>7f}")
    return phi

# Define B, w and c

In [6]:
NODES = 2
EDGES = 1
seed = 42

a = nx.adjacency_matrix(nx.gnm_random_graph(NODES, EDGES, seed=seed))

# a = np.array([[0, 1, 1], [1, 0, 1], [1, 1, 0]])
g = gx.GraphX(sp.sparse.csr_matrix.toarray(a))

DIM_CONNECTION = 2
h = cgx.ConnectionGraphX(sp.sparse.csr_matrix.toarray(a), DIM_CONNECTION)

In [7]:
B = h.connectionIncidenceMatrix.T.astype('float32')
w = np.ones(B.shape[0]//DIM_CONNECTION).astype('float32')

np.random.seed(42)
def rand_prob_mass(n, d):
    mu = np.random.uniform(0, 1, (n, d)).astype('float32')
    mu = mu/(mu.sum(axis=0)[None,:])
    mu = mu.flatten()[:,None]
    return mu

mu = rand_prob_mass(h.nNodes, DIM_CONNECTION)
nu = rand_prob_mass(h.nNodes, DIM_CONNECTION)
c = (mu - nu)

In [8]:
c.reshape((h.nNodes, DIM_CONNECTION))

array([[-0.39023045,  0.46100134],
       [ 0.39023048, -0.46100134]], dtype=float32)

In [9]:
c.reshape((h.nNodes, DIM_CONNECTION)).sum(axis=0)

array([2.9802322e-08, 0.0000000e+00], dtype=float32)

# Check feasibility of B

In [10]:
c_sol, residuals, _, _ = np.linalg.lstsq(B.T, c.flatten())

/var/folders/0d/431gq5hn0_ngj8qfl5bt02680000gn/T/ipykernel_22543/186593914.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  c_sol, residuals, _, _ = np.linalg.lstsq(B.T, c.flatten())


In [11]:
residuals

array([4.440892e-16], dtype=float32)

In [12]:
np.linalg.norm(c.flatten() - B.T.dot(c_sol).flatten())

2.9802322e-08

In [13]:
learning_rate = 0.01
alpha = 1e-5
n_epochs = 10000

B = torch.tensor(B)
w = torch.tensor(w)
c = torch.tensor(c)

optimize(B, w, c, alpha, learning_rate, n_epochs)

epoch: 0, loss: 112552.578125
epoch: 10, loss: 74600.328125
epoch: 20, loss: 45588.031250
epoch: 30, loss: 25352.365234
epoch: 40, loss: 12634.867188
epoch: 50, loss: 5530.144531
epoch: 60, loss: 2063.122559
epoch: 70, loss: 623.158081
epoch: 80, loss: 136.607285
epoch: 90, loss: 15.358568
epoch: 100, loss: -0.576125
epoch: 110, loss: -0.592404
epoch: 120, loss: -0.590870
epoch: 130, loss: -0.590312
epoch: 140, loss: -0.590110
epoch: 150, loss: -0.590038
epoch: 160, loss: -0.590014
epoch: 170, loss: -0.590006
epoch: 180, loss: -0.590005
epoch: 190, loss: -0.590006
epoch: 200, loss: -0.590008
epoch: 210, loss: -0.590010
epoch: 220, loss: -0.590012
epoch: 230, loss: -0.590014
epoch: 240, loss: -0.590017
epoch: 250, loss: -0.590019
epoch: 260, loss: -0.590021
epoch: 270, loss: -0.590024
epoch: 280, loss: -0.590027
epoch: 290, loss: -0.590029
epoch: 300, loss: -0.590032
epoch: 310, loss: -0.590035
epoch: 320, loss: -0.590038
epoch: 330, loss: -0.590041
epoch: 340, loss: -0.590043
epoch: 35

tensor([[-0.2005],
        [ 1.5188],
        [ 0.3453],
        [ 0.6809]], requires_grad=True)

# Working example

In [14]:
w = np.ones(1)
d = 2
sigma = np.eye(d)
B = np.block([np.sqrt(w)*np.eye(d), -np.sqrt(w)*sigma])

np.random.seed(42)
mu = np.random.uniform(0, 1, d)
mu = mu/mu.sum()
nu = np.random.uniform(0, 1, d)
nu = nu/nu.sum()
c = mu-nu
optim_val = np.linalg.norm(c)
optim_phi = c/np.linalg.norm(c)
c = np.block([c,-c])
c = c.reshape((2*d,1))

phi0 = np.block([optim_phi,optim_phi*0]).reshape(2*d,1)

print(B)
print(w)
print(c)
print(phi0)
print(optim_val)

B = torch.tensor(B.astype('float32'))
w = torch.tensor(w.astype('float32'))
c = torch.tensor(c.astype('float32'))
phi0 = phi0.astype('float32')

[[ 1.  0. -1. -0.]
 [ 0.  1. -0. -1.]]
[1.]
[[-0.26748401]
 [ 0.26748401]
 [ 0.26748401]
 [-0.26748401]]
[[-0.70710678]
 [ 0.70710678]
 [-0.        ]
 [ 0.        ]]
0.37827952162613965


# Solve Beckmann - connection problem

In [15]:
learning_rate = 0.001
alpha = 1e-1
n_epochs = 10000

In [16]:
optimize(B, w, c, alpha, learning_rate, n_epochs, phi0=phi0)

epoch: 0, loss: -0.378280
epoch: 10, loss: -0.384847
epoch: 20, loss: -0.385245
epoch: 30, loss: -0.385234
epoch: 40, loss: -0.385434
epoch: 50, loss: -0.385405
epoch: 60, loss: -0.385434
epoch: 70, loss: -0.385430
epoch: 80, loss: -0.385434
epoch: 90, loss: -0.385434
epoch: 100, loss: -0.385434
epoch: 110, loss: -0.385434
epoch: 120, loss: -0.385434
epoch: 130, loss: -0.385434
epoch: 140, loss: -0.385434
epoch: 150, loss: -0.385434
epoch: 160, loss: -0.385434
epoch: 170, loss: -0.385434
epoch: 180, loss: -0.385434
epoch: 190, loss: -0.385434
epoch: 200, loss: -0.385434
epoch: 210, loss: -0.385434
epoch: 220, loss: -0.385434
epoch: 230, loss: -0.385434
epoch: 240, loss: -0.385434
epoch: 250, loss: -0.385434
epoch: 260, loss: -0.385434
epoch: 270, loss: -0.385434
epoch: 280, loss: -0.385434
epoch: 290, loss: -0.385434
epoch: 300, loss: -0.385434
epoch: 310, loss: -0.385434
epoch: 320, loss: -0.385434
epoch: 330, loss: -0.385434
epoch: 340, loss: -0.385434
epoch: 350, loss: -0.385434
epo

tensor([[-0.7205],
        [ 0.7205],
        [ 0.0134],
        [-0.0134]], requires_grad=True)

# Attempt Beckmann on Puppet Connection Graph

Initialize the puppet connection graph

In [ ]:
p = "/Users/sawyerjack/IdeaProjects/project_data/PuppetsData"

N_PUPPET_IMAGES = 500
NEAREST_NEIGHBORS = 7
INTRINSIC_DIMENSION = 2

puppetGraph = cnx.cnxFromImageDirectory(p, INTRINSIC_DIMENSION, k=NEAREST_NEIGHBORS, nImages=N_PUPPET_IMAGES)


Setting up B, c, w for the puppet graph

In [ ]:
puppetB = puppetGraph.connectionIncidenceMatrix.T.astype('float32')
puppetW = np.ones(puppetB.shape[0]//INTRINSIC_DIMENSION).astype('float32')

np.random.seed(42)

#puppetMu = rand_prob_mass(puppetGraph.nNodes, INTRINSIC_DIMENSION)
#puppetNu = rand_prob_mass(puppetGraph.nNodes, INTRINSIC_DIMENSION)

puppetMu = np.zeros((puppetGraph.nNodes * INTRINSIC_DIMENSION,1))
puppetMu[0:2, :] = [[1],[1]]
puppetNu = np.zeros((puppetGraph.nNodes * INTRINSIC_DIMENSION,1))
puppetNu[2:4, :] = [[1],[1]]

puppetC = (puppetMu - puppetNu)

puppetC.reshape((puppetGraph.nNodes, INTRINSIC_DIMENSION))
puppetC.reshape((puppetGraph.nNodes, INTRINSIC_DIMENSION)).sum(axis=0)


Checking Feasibility - looks good

In [ ]:
print(type(puppetC))
print(type(puppetB))
c_sol, residuals, _, _ = np.linalg.lstsq(sp.sparse.lil_matrix.toarray(puppetB.T), puppetC)

# I didn't rename the dummy variables here- caution.

np.linalg.norm(puppetC.flatten() - puppetB.T.dot(c_sol).flatten())
#np.linalg.norm(c_sol)


Attempt to plug into the optimization methods

In [ ]:
learning_rate = 0.1
alpha = 1e-5
n_epochs = 10000

B = torch.tensor(sp.sparse.lil_matrix.toarray(puppetB))
w = torch.tensor(puppetW)
c = torch.tensor(puppetC)

puppetPhi = optimize(B, w, c, alpha, learning_rate, n_epochs)

In [ ]:
np.linalg.norm(puppetPhi.detach().numpy())

Doing some post-hoc illustration- want to see "active edges"

In [ ]:
#print(c_sol.shape)

#for edge in range(1846):
#    if abs(c_sol[edge]) < TOLERANCE:
#        c_sol[edge] = 0

# with np.printoptions(threshold=np.inf):
    # print(c_sol)


In [ ]:
import puppets_data
Y_, labelsMat_, _ = puppets_data.puppets_data(p)

In [ ]:
from umap import UMAP
umap_obj = UMAP(n_neighbors=13, min_dist=0.5, n_components=3, random_state=42)
umap_puppetGraph_embedding = umap_obj.fit_transform(puppetGraph.imageData)



In [ ]:
umap_puppetGraph_embedding[0,2]

In [ ]:
from mpl_toolkits.mplot3d.art3d import Line3D

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# ax.set_xlim((-1.1, 1.1))
# ax.set_ylim((-1.1, 1.1))
# ax.set_zlim((-1.1, 1.1))

nodeData = nx.kamada_kawai_layout(puppetGraph)

TOLERANCE = 1e-4
MAX = np.amax(abs(c_sol))

for edgeIndex, edge in zip(range(puppetGraph.nEdges), list(puppetGraph.edges())):

    if abs(c_sol[edgeIndex])[0] > TOLERANCE:
        col=(1, 0, 0, abs(c_sol[edgeIndex])[0]/MAX)
        z = 3
    else:
        #col="tab:blue"
        col=(1,1,1,0)
        z = 1

    fromNode = edge[0]
    toNode = edge[1]
    centerFromNode = umap_puppetGraph_embedding[fromNode, :]
    centerToNode = umap_puppetGraph_embedding[toNode, :]

    ax.plot((centerFromNode[0], centerToNode[0]), (centerFromNode[1], centerToNode[1]), (centerFromNode[2], centerToNode[2]), color=col, lw=3)
    # edgeLabel = Line3D((centerFromNode[0], centerToNode[0]), (centerFromNode[1], centerToNode[1]), (centerFromNode[2], centerToNode[2]), color=col, lw=3)
    # edgeLabel.zorder = z
    # ax.add_line(edgeLabel)

# for node in tqdm(list(range(2, N_PUPPET_IMAGES)) + [0, 1]):
#     center = nodeData[node][0], nodeData[node][1]
#     if node in [0,1]:
#         col="tab:red"
#         zNode=2
#         r=1e-1
#         #ax.text(center[0], center[1], str(node+1))
#     else:
#         col="tab:blue"
#         zNode=1
#         r=2e-2
#     nodeLabel = matplotlib.patches.Circle(center, radius=r, color=col, zorder=zNode)
#     ax.add_patch(nodeLabel)

nodeColor = np.zeros(N_PUPPET_IMAGES)
nodeColor[0:2] = 1

ax.scatter(*umap_puppetGraph_embedding.T, c='k', marker='o')

ax.scatter(*umap_puppetGraph_embedding[0:2,:].T, c='r', s=30, marker='*')
plt.show()
# Want to color "active" edges red. WIP.
# Noticed that CNX has an issue- not properly removing the remote edges. Zeroing the Connection incidence is not enough.


In [ ]:
print(puppetGraph.nEdges * 2)
print(c_sol.shape)
print(puppetPhi.detach().numpy().shape)



In [ ]:
np.amax(c_sol)